In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import urllib.request
import json
import os
import cartopy.crs as ccrs
import pytide

from pathlib import Path

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension("bokeh")

from bokeh.io import curdoc, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.themes import built_in_themes

output_notebook()
curdoc().theme = "light_minimal"

In [ ]:
# project files
d3d = Path("/mnt/c/Users/rdchlclj/Projects/MR_D3D_model/Delft3D")
data = Path("/mnt/e/MS_River_plume")
project = d3d / "tidal_constituent_boundary_conditions"
output = project / "output"
figures = project / "figures"

# NOAA COOPs
const_data = project / "constituent_data"
noaa_prediction_data = project / "noaa_COOPs_prediction_data"
tidal_stations_fn = Path(
    "/mnt/e/MS_River_plume/ArcPro/MyProject/output data/tidal_constituents_stations.xlsx"
)

tidal_stations_all = pd.read_excel(tidal_stations_fn, index_col=[0])

In [ ]:
class FMmodel:
    def __init__(self, case, case_name, study, model_output):
        self.case = case
        self.case_name = case_name
        self.study = study
        self.model_output = model_output
        self.his_data = self.load_his(self.model_output / f"{self.case}_0000_his.nc")

    def load_his(self, his_fn):
        return (
            xr.open_dataset(his_fn)
            .swap_dims({"stations": "station_name"})
            .drop_vars(["station_id"])
        )

    def create_comparison_data(self, stations):
        wl_dataset = {}

        for station in stations:
            station_name = station.encode()
            data = (
                self.his_data["waterlevel"]
                .sel(station_name=station_name)
                .to_dataframe()["waterlevel"]
            )
            data.rename()
            wl_dataset[station] = data

        self.wl_dataset = wl_dataset
        self.wl_curves = {
            station: hv.Curve(wl, group=self.case, label=self.case)
            for station, wl in wl_dataset.items()
        }
    
    def demean_wl(self):
        
        self.demeaned_wl_dataset = {}
        
        for key, wl in self.wl_dataset.items():
            mean_wl = wl.mean()
            self.demeaned_wl_dataset[key] = wl - mean_wl
        
        self.demeaned_wl_curves = {
            station: hv.Curve(wl, group=self.case, label=self.case)
            for station, wl in self.demeaned_wl_dataset.items()
        }

# Observations

In [ ]:
# clean up data
# tidal stations to drop
drop_stations = (
    "Pensacola",
    "East Fowl River Bridge",
    "Mobile State Docks",
    "Chickasaw Creek",
    "West Fowl River Bridge",
    "Bayou La Batre Bridge",
    "Grand Bay NERR, Mississippi Sound",
    "Pascagoula NOAA Lab",
    "Sabine Pass North",
    "Texas Point, Sabine Pass",
    "Weeks Bay, Mobile Bay",
)
tidal_stations = tidal_stations_all[~tidal_stations_all.name.isin(drop_stations)]

no_NAVD = [
    "Eugene_Island_North_of__Gulf_of_Mexico",
    "I-10_Bonnet_Carre_Floodway",
    "Port_Fourchon_Belle_Pass",
    "Grand_Isle",
    "Pilots_Station_East_S.W._Pass",
    "Pilottown",
    "Dog_River_Bridge",
]

In [ ]:
# NOAA time series data
begin_date = "20180101"
end_date = "20190101"
product = "predictions"
interval = "h"
datum = "NAVD"
form = "csv"
time_zone = "gmt"
units = "metric"

noaa_predicted_time_series = {}

for _, (station_id, name) in tidal_stations[["id", "name"]].iterrows():

    # meta data
    station_code = name.replace(" ", "_").replace(",", "")
    
    if station_code in no_NAVD:
        continue
    
    request = f"https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin_date}&end_date={end_date}&station={station_id}&product={product}&datum={datum}&units={units}&time_zone={time_zone}&application=ERDC&format={form}&interval={interval}"
    out_fn = noaa_prediction_data / f"{station_code}_{product}_ts_{begin_date}-{end_date}_{datum}.csv"

    # conditionally download
    if not out_fn.exists():
        csv, http = urllib.request.urlretrieve(request, out_fn)
        pass
    else:
        csv = out_fn
        pass
    
    noaa_data = pd.read_csv(
        csv,
        index_col=[0],
        parse_dates=True,
        names=["time", "prediction"],
        header=0,
        usecols=[0, 1]
    )
    noaa_predicted_time_series[station_code] = noaa_data.squeeze()

In [ ]:
# NOAA time series
begin_date = "20180101"
end_date = "20190101"
product = "predictions"
interval = "h"
datum = "MSL"
form = "csv"
time_zone = "gmt"
units = "metric"

noaa_predicted_time_series_MSL = {}

for _, (station_id, name) in tidal_stations[["id", "name"]].iterrows():

    # meta data
    station_code = name.replace(" ", "_").replace(",", "")
    request = f"https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin_date}&end_date={end_date}&station={station_id}&product={product}&datum={datum}&units={units}&time_zone={time_zone}&application=ERDC&format={form}&interval={interval}"
    out_fn = noaa_prediction_data / f"{station_code}_{product}_ts_{begin_date}-{end_date}_{datum}.csv"

    # conditionally download
    if not out_fn.exists():
        csv, http = urllib.request.urlretrieve(request, out_fn)
        pass
    else:
        csv = out_fn
        pass

    noaa_data = pd.read_csv(
        csv,
        index_col=[0],
        parse_dates=True,
        names=["time", "prediction"],
        header=0,
        usecols=[0, 1],
    )
    noaa_predicted_time_series_MSL[station_code] = noaa_data.squeeze()

In [ ]:
# NOAA observed time series data
begin_date = "20180101"
end_date = "20190101"
product = "hourly_height"
interval = "h"
datum = "NAVD"
form = "csv"
time_zone = "gmt"
units = "metric"

noaa_measured_time_series = {}

for _, (station_id, name) in tidal_stations[["id", "name"]].iterrows():

    # meta data
    station_code = name.replace(" ", "_").replace(",", "")
    
    if station_code in no_NAVD:
        continue
    
    request = f"https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin_date}&end_date={end_date}&station={station_id}&product={product}&datum={datum}&units={units}&time_zone={time_zone}&application=ERDC&format={form}"
    out_fn = noaa_prediction_data / f"{station_code}_{product}_ts_{begin_date}-{end_date}_{datum}.csv"

    # conditionally download
    if not out_fn.exists():
        csv, http = urllib.request.urlretrieve(request, out_fn)
        pass
    else:
        csv = out_fn
        pass

    noaa_data = pd.read_csv(
        csv,
        index_col=[0],
        parse_dates=True,
        names=["time", "waterlevel"],
        header=0,
        usecols=[0, 1],
    )
    noaa_measured_time_series[station_code] = noaa_data.squeeze()

In [ ]:
# NOAA observed time series data
begin_date = "20180101"
end_date = "20190101"
product = "hourly_height"
interval = "h"
datum = "MSL"
form = "csv"
time_zone = "gmt"
units = "metric"

noaa_measured_time_series_MSL = {}

for _, (station_id, name) in tidal_stations[["id", "name"]].iterrows():

    # meta data
    station_code = name.replace(" ", "_").replace(",", "")
    request = f"https://tidesandcurrents.noaa.gov/api/datagetter?begin_date={begin_date}&end_date={end_date}&station={station_id}&product={product}&datum={datum}&units={units}&time_zone={time_zone}&application=ERDC&format={form}"
    out_fn = noaa_prediction_data / f"{station_code}_{product}_ts_{datum}.csv"

    # conditionally download
    if not out_fn.exists():
        csv, http = urllib.request.urlretrieve(request, out_fn)
        pass
    else:
        csv = out_fn
        pass

    noaa_data = pd.read_csv(
        csv,
        index_col=[0],
        parse_dates=True,
        names=["time", "waterlevel"],
        header=0,
        usecols=[0, 1],
    )
    
    noaa_measured_time_series_MSL[station_code] = noaa_data.squeeze()

In [ ]:
# observation stations
stations = list(noaa_measured_time_series_MSL.keys())

# combine
reference_values = {
    "predicted": noaa_predicted_time_series,
    "predicted_MSL": noaa_predicted_time_series_MSL,
    "verified": noaa_measured_time_series,
    "verified_MSL": noaa_measured_time_series_MSL,
}

# Model results

In [ ]:
# load model results
model_ref = pd.to_datetime('2018-01-01 00:00:00')
study = "roughness_calibration"

case = "r01"
case_name = f"{case}_10sig_adcirc11"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

r01 = FMmodel(case, case_name, study, model_output)
r01.create_comparison_data(stations)
r01.demean_wl()

In [ ]:
# load model results
model_ref = pd.to_datetime('2018-01-01 00:00:00')
study = "roughness_calibration"

case = "r02"
case_name = f"{case}_10sig_default_0023"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

r02 = FMmodel(case, case_name, study, model_output)
r02.create_comparison_data(stations)
r02.demean_wl()

In [ ]:
# load model results
model_ref = pd.to_datetime('2018-01-01 00:00:00')
study = "tidal_calibration"

case = "td_01"
case_name = f"{case}_1sig_base"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

td_01 = FMmodel(case, case_name, study, model_output)
td_01.create_comparison_data(stations)
td_01.demean_wl()

In [ ]:
# load model results
model_ref = pd.to_datetime('2038-03-03 00:00:00')
study = "tidal_calibration"

case = "td03"
case_name = f"{case}_1sig_rough"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

td03 = FMmodel(case, case_name, study, model_output)
td03.create_comparison_data(stations)
td03.demean_wl()

In [ ]:
# load model results
model_ref = pd.to_datetime('2018-01-01 00:00:00')
study = "tidal_calibration"

case = "t11"
case_name = f"{case}_solar_corr"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

t11 = FMmodel(case, case_name, study, model_output)
t11.create_comparison_data(stations)
t11.demean_wl()

In [ ]:
# load model results
model_ref = pd.to_datetime('2018-01-01 00:00:00')
study = "three_dimensional"

case = "f07"
case_name = f"{case}_10sig_davg_sal_rst"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

f07 = FMmodel(case, case_name, study, model_output)
f07.create_comparison_data(stations)
f07.demean_wl()

In [ ]:
study = "meteo_testing"

case = "m01"
case_name = f"{case}_10sig_default"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

m01 = FMmodel(case, case_name, study, model_output)
m01.create_comparison_data(stations)
m01.demean_wl()

case = "m02"
case_name = f"{case}_10sig_cfsv2"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

m02 = FMmodel(case, case_name, study, model_output)
m02.create_comparison_data(stations)
m02.demean_wl()

case = "m03"
case_name = f"{case}_10sig_ncar_rean2"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

m03 = FMmodel(case, case_name, study, model_output)
m03.create_comparison_data(stations)
m03.demean_wl()

case = "m04"
case_name = f"{case}_10sig_era5"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

m04 = FMmodel(case, case_name, study, model_output)
m04.create_comparison_data(stations)
m04.demean_wl()

case = "m05"
case_name = f"{case}_10sig_era5_elias"
model_output = data / f"Delft3d/models/{study}/{case_name}/output"

m05 = FMmodel(case, case_name, study, model_output)
m05.create_comparison_data(stations)
m05.demean_wl()

In [ ]:
kdims = hv.Dimension("station", default="Dauphin_Island")

holomap_obs = hv.HoloMap(
    {
        station: hv.Curve(wl, group="obs", label="obs")
        for station, wl in reference_values["verified_MSL"].items()
    },
    kdims=kdims,
    label="obs",
)

holomap_pred = hv.HoloMap(
    {
        station: hv.Curve(wl, group="pred", label="pred.")
        for station, wl in reference_values["predicted_MSL"].items()
    },
    kdims=kdims,
    label="pred.",
)

In [ ]:
holomap4 = hv.HoloMap(td_01.demeaned_wl_curves, kdims=kdims)
holomap5 = hv.HoloMap(t11.demeaned_wl_curves, kdims=kdims)
holomap6 = hv.HoloMap(td03.demeaned_wl_curves, kdims=kdims)
overlay = holomap4 * holomap6 * holomap_pred
overlay.opts(
    opts.Curve(width=1400, height=900),
    opts.Curve("pred", color="blue"),
    opts.Curve("td_01", color="black"),
    opts.Curve("t11", color="red"),
    opts.Curve("td03", color="green")
).opts(
    show_grid=True,
    xlim=(pd.to_datetime("2018-01-15"), pd.to_datetime("2018-02-15")),
    ylim=(-1, 1),
)

In [ ]:
holomap1 = hv.HoloMap(m04.demeaned_wl_curves, kdims=kdims)
holomap2 = hv.HoloMap(m05.demeaned_wl_curves, kdims=kdims)
holomap3 = hv.HoloMap(f07.demeaned_wl_curves, kdims=kdims)

overlay = holomap1 * holomap2 * holomap_obs
overlay.opts(
    opts.Curve(width=1400, height=900),
    opts.Curve("m04", color="red"),
    opts.Curve("m05", color="black"),
    opts.Curve("obs", color="blue", line_dash="solid"),
).opts(
    show_grid=True,
    xlim=(pd.to_datetime("2018-01-15"), pd.to_datetime("2018-02-15")),
    ylim=(-1, 1),
)

In [ ]:
holomap_r01 = hv.HoloMap(r01.demeaned_wl_curves, kdims=kdims)
holomap_r02 = hv.HoloMap(r02.demeaned_wl_curves, kdims=kdims)

overlay = holomap_r01 * holomap_r02 * holomap_obs
overlay.opts(
    opts.Curve(width=1400, height=900),
    opts.Curve("r02", color="red"),
    opts.Curve("r01", color="black"),
    opts.Curve("obs", color="blue", line_dash="solid"),
).opts(
    show_grid=True,
    xlim=(pd.to_datetime("2018-01-15"), pd.to_datetime("2018-02-15")),
    ylim=(-1, 1),
)

In [ ]:
holomap_m01 = hv.HoloMap(m01.demeaned_wl_curves, kdims=kdims)
holomap_m02 = hv.HoloMap(m02.demeaned_wl_curves, kdims=kdims)
holomap_m03 = hv.HoloMap(m03.demeaned_wl_curves, kdims=kdims)
holomap_m04 = hv.HoloMap(m04.demeaned_wl_curves, kdims=kdims)
holomap_m05 = hv.HoloMap(m05.demeaned_wl_curves, kdims=kdims)

overlay = holomap_m01 * holomap_m02 * holomap_m03 * holomap_m04 * holomap_m05 * holomap_obs
overlay.opts(
    opts.Curve(width=1400, height=900),
    opts.Curve("m02", color="red"),
    opts.Curve("m01", color="black"),
    opts.Curve("m03", color="green"),
    opts.Curve("m04", color="yellow"),
    opts.Curve("m05", color="purple"),
    opts.Curve("obs", color="blue", line_dash="solid"),
).opts(
    show_grid=True,
    xlim=(pd.to_datetime("2018-01-15"), pd.to_datetime("2018-02-15")),
    ylim=(-1, 1),
)